## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import collections


In [2]:
df= pd.read_csv('data/sample_2019.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fl_date              12000 non-null  object 
 1   mkt_unique_carrier   12000 non-null  object 
 2   branded_code_share   12000 non-null  object 
 3   mkt_carrier          12000 non-null  object 
 4   mkt_carrier_fl_num   12000 non-null  int64  
 5   op_unique_carrier    12000 non-null  object 
 6   tail_num             11955 non-null  object 
 7   op_carrier_fl_num    12000 non-null  int64  
 8   origin_airport_id    12000 non-null  int64  
 9   origin               12000 non-null  object 
 10  origin_city_name     12000 non-null  object 
 11  dest_airport_id      12000 non-null  int64  
 12  dest                 12000 non-null  object 
 13  dest_city_name       12000 non-null  object 
 14  crs_dep_time         12000 non-null  int64  
 15  dep_time             11772 non-null 

In [4]:
df['fl_month']= df.fl_date.str.split("-",expand=True).iloc[:,1]

In [5]:
# dataset of 1000 random row in each month in 2019
df.fl_month.value_counts()

01    1000
02    1000
03    1000
04    1000
05    1000
06    1000
07    1000
08    1000
09    1000
10    1000
11    1000
12    1000
Name: fl_month, dtype: int64

#### **Task 1**: Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

In [6]:
# 33% of flights was departure delayed
df[df.dep_delay>0].shape[0]/df.shape[0]

0.333

In [7]:
# 23% of flight are arrival delays caused by departure delays
df[(df.dep_delay>0) & (df.arr_delay>0)].shape[0]/df.shape[0]

0.23925

In [8]:
# 9% of flight are delayed in departure but arrive ontime or early
df[(df.dep_delay>0) & (df.arr_delay<=0)].shape[0]/df.shape[0]

0.091

33% of flights was departure delayed
- 9% of flight which was delay in departure arrive on time (or even earlier)
- 23% delay in both departure and arrival

#### **Task 6**: How many states cover 50% of US air traffic? 

In [9]:
df.origin_city_name.head()

0      Chicago, IL
1      Houston, TX
2      Buffalo, NY
3    San Diego, CA
4      Houston, TX
Name: origin_city_name, dtype: object

In [10]:
# state in origin airport (last 2 letters)
origin_state= df.origin_city_name.str.split(pat=", ",expand=True).iloc[:,1].to_list()
len(origin_state)

12000

In [11]:
df.dest_city_name.head()

0         Washington, DC
1     Corpus Christi, TX
2         Washington, DC
3          Las Vegas, NV
4    Fort Lauderdale, FL
Name: dest_city_name, dtype: object

In [12]:
# state in destination airport, last 2 letters
destination_state= df.dest_city_name.str.split(pat=", ",expand=True).iloc[:,1].to_list()

In [13]:
air_traffic= origin_state + destination_state
len(air_traffic)

24000

In [14]:
result= dict(collections.Counter(air_traffic))
result

{'IL': 1575,
 'TX': 2439,
 'NY': 1271,
 'CA': 2589,
 'NJ': 466,
 'OH': 532,
 'DC': 672,
 'WA': 723,
 'PA': 697,
 'AL': 128,
 'GA': 1236,
 'CO': 976,
 'TN': 436,
 'AZ': 621,
 'NC': 1115,
 'KY': 131,
 'IA': 93,
 'FL': 1726,
 'MD': 327,
 'MI': 657,
 'IN': 237,
 'MO': 441,
 'NE': 103,
 'LA': 263,
 'HI': 305,
 'MA': 498,
 'AK': 132,
 'UT': 370,
 'CT': 97,
 'VA': 233,
 'NV': 551,
 'MN': 519,
 'OR': 336,
 'OK': 142,
 'AR': 101,
 'ID': 101,
 'WI': 199,
 'PR': 87,
 'VT': 45,
 'KS': 54,
 'SC': 208,
 'NM': 97,
 'ME': 64,
 'ND': 78,
 'MT': 90,
 'RI': 49,
 'MS': 42,
 'WV': 28,
 'WY': 27,
 'TT': 2,
 'SD': 48,
 'VI': 11,
 'NH': 32}

In [15]:
result= pd.Series(result,index=result.keys()).reset_index()
result.head()

,index,0
0,IL,1575
1,TX,2439
2,NY,1271
3,CA,2589
4,NJ,466


In [16]:
# change column name and calculate percentage
result.columns=['State','nbr_flights']
result['percentage']= (result.nbr_flights/10000*100).round(2)

In [17]:
result.shape
# there are 53 states with flight operation

(53, 3)

In [18]:
result.sort_values(by='percentage', ascending=False).head(5)
# 2 States cover 50% of US air traffic

,State,nbr_flights,percentage
3,CA,2589,25.89
1,TX,2439,24.39
17,FL,1726,17.26
0,IL,1575,15.75
2,NY,1271,12.71


out of 52 State, 2 States cover 50% of US air traffic

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

- Short-haul is a flight lasting anywhere from 30 minutes to 3 hours
- Medium-haul is defined by flights lasting between 3-6 hours
- And lastly, long-haul flights are those that extend beyond 6 hours.

In [19]:
df.air_time.describe().round(0)

count    11716.0
mean       108.0
std         70.0
min         10.0
25%         58.0
50%         90.0
75%        136.0
max        547.0
Name: air_time, dtype: float64

In [20]:
# classified type of flight based on flight duration
df['flight_type']= np.where(df.air_time <180, "SHORT", np.where(df.air_time<360,"MEDIUM","LONG"))

In [21]:
df[['air_time','flight_type','dep_time']].head()

,air_time,flight_type,dep_time
0,84.0,SHORT,1353.0
1,45.0,SHORT,1424.0
2,59.0,SHORT,1024.0
3,55.0,SHORT,1709.0
4,119.0,SHORT,2003.0


In [28]:
df.flight_type.value_counts()
# majority of flight are short_haul

SHORT     10120
MEDIUM     1544
LONG        336
Name: flight_type, dtype: int64

In [22]:
# take hour of departure
df['dep_hour']= np.floor(df.dep_time/100)

In [27]:
df[['flight_type','dep_time','dep_hour']].head(10)

,flight_type,dep_time,dep_hour
0,SHORT,1353.0,13.0
1,SHORT,1424.0,14.0
2,SHORT,1024.0,10.0
3,SHORT,1709.0,17.0
4,SHORT,2003.0,20.0
5,SHORT,658.0,6.0
6,SHORT,1210.0,12.0
7,SHORT,1225.0,12.0
8,SHORT,713.0,7.0
9,SHORT,1925.0,19.0


In [24]:
# for each flight type, count number of flight by dep_hour
take_off_hour= df.groupby(['flight_type','dep_hour']).fl_date.count().reset_index()
take_off_hour.columns=['flight_type',"dep_hour","nbr_flight"]

In [25]:
take_off_hour.head()

,flight_type,dep_hour,nbr_flight
0,LONG,5.0,2
1,LONG,6.0,7
2,LONG,7.0,7
3,LONG,8.0,1
4,LONG,9.0,5


In [31]:
take_off_hour[take_off_hour.flight_type =="SHORT"].sort_values('nbr_flight',ascending=False)
# short-haul flight are take off in the afternoon 

,flight_type,dep_hour,nbr_flight
61,SHORT,17.0,640
51,SHORT,7.0,634
55,SHORT,11.0,631
58,SHORT,14.0,623
59,SHORT,15.0,622
57,SHORT,13.0,618
62,SHORT,18.0,617
56,SHORT,12.0,617
50,SHORT,6.0,616
52,SHORT,8.0,594


In [33]:
take_off_hour[take_off_hour.flight_type =="MEDIUM"].sort_values('nbr_flight',ascending=False)
# Medium flight often take off in the morning

,flight_type,dep_hour,nbr_flight
26,MEDIUM,7.0,124
25,MEDIUM,6.0,118
27,MEDIUM,8.0,115
29,MEDIUM,10.0,97
30,MEDIUM,11.0,94
34,MEDIUM,15.0,90
37,MEDIUM,18.0,86
33,MEDIUM,14.0,84
31,MEDIUM,12.0,79
36,MEDIUM,17.0,78


In [34]:
take_off_hour[take_off_hour.flight_type =="LONG"].sort_values('nbr_flight',ascending=False)
# Medium flight often take off in the morning

,flight_type,dep_hour,nbr_flight
12,LONG,17.0,12
10,LONG,15.0,9
5,LONG,10.0,9
18,LONG,23.0,8
13,LONG,18.0,8
7,LONG,12.0,7
1,LONG,6.0,7
9,LONG,14.0,7
2,LONG,7.0,7
8,LONG,13.0,6


#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?